In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "wireshark", 2

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'epan/wslua/wslua_tvb.c'}
bug.buggy_lines={('epan/wslua/wslua_tvb.c', 1404), ('epan/wslua/wslua_tvb.c', 1403), ('epan/wslua/wslua_tvb.c', 1402), ('epan/wslua/wslua_tvb.c', 1405), ('epan/wslua/wslua_tvb.c', 1401)}
bug.coverage_df=


23  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   432   
                                                                                  177   432   
                                                                                  199   432   
                                                                                  203   432   
                                                                                  205   432   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1720  432   
                                                                                  1721  432   
                                                                                  1723  432   
                                                                                  1724  432   
                                                                                  1725  432   

                                                                                         26  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   451   
                                                                                  177   451   
                                                                                  199   451   
                                                                                  203   451   
                                                                                  205   451   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1720  451   
                                                                                  1721  451   
                                                                                  1723  451   
                                                                                  1724  451   
                                                                                  1725  451   

                                                                                         30  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   467   
                                                                                  177   467   
                                                                                  199   467   
                                                                                  203   467   
                                                                                  205   467   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1720  467   
                                                                                  1721  467   
                                                                                  1723  467   
                                                                                  1724  467   
                                                                                  1725  467   

                                                                                         19  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   402   
                                                                                  177   402   
 

bug.failing_tests={'34'}
bug.failing_tests_GT={'34'}
bug.functions={('/home/workspace/epan/dissectors/packet-bthci_vendor.c', 'proto_reg_handoff_bthci_vendor_intel'), ('/home/workspace/tools/lemon/lemon.c', 'preprocess_input'), ('/home/workspace/epan/tvbuff.c', 'tvb_get_letohs'), ('/home/workspace/epan/dissectors/packet-zbee-zcl-ha.c', 'proto_register_zbee_zcl_appl_stats'), ('/home/workspace/epan/dissectors/packet-ubertooth.c', 'proto_reg_handoff_ubertooth'), , ...}
bug.id='wireshark-2'
bug.num_test_cases=34
bug.passing_tests={'7', '1', '5', '11', '31', '24', '20', '21', '32', '6', '26', '12', '14', '27', '16', '29', '22', '10', '9', '25', '4', '15', '33', '19', '18', '2', '13', '3', '28', '8', '23', '17', '30'}
bug.path_to_cov_df='/home/coinse/Workspace/defects4cpp/coverage/wireshark_2_buggy/coverage_df.pkl'
bug.path_to_patch='/home/coinse/Workspace/defects4cpp/defects4cpp/taxonomy/wireshark/patch/0002-buggy.patch'
bug.path_to_test_results='/home/coinse/Workspace/defects4cpp/coverage/

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                                  function                    line                                                                                              
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 174)   
                                                                                  177   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 177)   
                                                                                  199   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 199)   
                                                                                  203   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 203)   
                                                                                  205   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 205)   
...                                                                                                                                                                           ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1720  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1720)               
                                                                                  1721  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1721)               
                                                                                  1723  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1723)               
                                                                                  1724  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1724)               
                                                                                  1725  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1725)               

                                                                                        e_p  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   33    
                                                                                  177   33    
                                                                                  199   33    
                                                                                  203   33    
                                                                                  205   33    
...                                                                                     ..    
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1720  33    
                                                                                  1721  33    
                                                                                  1723  33    
                                                                                  1724  33    
                                                                                  1725  33    

                                                                                        e_f  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   1     
                                                                                  177   1     
                                                                                  199   1     
                                                                                  203   1     
             

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/epan/wslua/wslua_tvb.c', 'TvbRange__tostring')
fault idx: ('/home/workspace/epan/wslua/wslua_tvb.c', 'TvbRange__tostring', 1402)
fault idx: ('/home/workspace/epan/wslua/wslua_tvb.c', 'TvbRange__tostring', 1404)
fault idx: ('/home/workspace/epan/wslua/wslua_tvb.c', 'TvbRange__tostring', 1405)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring, 1402)",0,1,33,0,1.0,1.0,5026.0,5026.0
1,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring, 1404)",0,1,33,0,1.0,1.0,5026.0,5026.0
2,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring, 1405)",0,1,33,0,1.0,1.0,5026.0,5026.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/wslua/wslua_file_handler.c, wslua_filehandler_open, 183)",0,1,33,0,1.0,1.0,5026.0,5026.0
1,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7773)",0,1,33,0,1.0,1.0,5026.0,5026.0
2,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7785)",0,1,33,0,1.0,1.0,5026.0,5026.0
3,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7784)",0,1,33,0,1.0,1.0,5026.0,5026.0
4,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7780)",0,1,33,0,1.0,1.0,5026.0,5026.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/wslua/wslua_file_handler.c, wslua_filehandler_open, 183)",0,1,33,0,1.0,1.0,5026.0,5026.0
1,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7773)",0,1,33,0,1.0,1.0,5026.0,5026.0
2,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7785)",0,1,33,0,1.0,1.0,5026.0,5026.0
3,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7784)",0,1,33,0,1.0,1.0,5026.0,5026.0
4,"(/home/workspace/epan/proto.c, proto_free_field_strings, 7780)",0,1,33,0,1.0,1.0,5026.0,5026.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,174,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
1,177,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
2,199,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
3,203,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
4,205,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
...,...,...,...,...,...,...,...,...,...,...,...,...
96698,1720,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
96699,1721,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
96700,1723,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
96701,1724,33,1,0,0,0.029412,0.171499,96593.0,96593.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1407,0,1,33,0,1.0,677.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/wsutil/time_util.c, mktime_utc)",58,0,1,33,0,1.0,677.0
1,"(/home/workspace/wsutil/report_message.c, report_failure)",54,0,1,33,0,1.0,677.0
2,"(/home/workspace/wsutil/nstime.c, nstime_sum)",127,0,1,33,0,1.0,677.0
3,"(/home/workspace/wsutil/filesystem.c, get_dirname)",154,0,1,33,0,1.0,677.0
4,"(/home/workspace/wiretap/wtap.c, wtap_sequential_close)",1207,0,1,33,0,1.0,677.0
5,"(/home/workspace/wiretap/visual.c, visual_open)",175,0,1,33,0,1.0,677.0
6,"(/home/workspace/wiretap/tnef.c, tnef_open)",24,0,1,33,0,1.0,677.0
7,"(/home/workspace/wiretap/snoop.c, snoop_open)",249,0,1,33,0,1.0,677.0
8,"(/home/workspace/wiretap/rfc7468.c, rfc7468_open)",80,0,1,33,0,1.0,677.0
9,"(/home/workspace/wiretap/rfc7468.c, read_complete_text_line)",41,0,1,33,0,1.0,677.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1407,0,1,33,0,1.0,677.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/wsutil/time_util.c, mktime_utc)",58,0,1,33,0,1.0,677.0
1,"(/home/workspace/wsutil/report_message.c, report_failure)",54,0,1,33,0,1.0,677.0
2,"(/home/workspace/wsutil/nstime.c, nstime_sum)",127,0,1,33,0,1.0,677.0
3,"(/home/workspace/wsutil/filesystem.c, get_dirname)",154,0,1,33,0,1.0,677.0
4,"(/home/workspace/wiretap/wtap.c, wtap_sequential_close)",1207,0,1,33,0,1.0,677.0
5,"(/home/workspace/wiretap/visual.c, visual_open)",175,0,1,33,0,1.0,677.0
6,"(/home/workspace/wiretap/tnef.c, tnef_open)",24,0,1,33,0,1.0,677.0
7,"(/home/workspace/wiretap/snoop.c, snoop_open)",249,0,1,33,0,1.0,677.0
8,"(/home/workspace/wiretap/rfc7468.c, rfc7468_open)",80,0,1,33,0,1.0,677.0
9,"(/home/workspace/wiretap/rfc7468.c, read_complete_text_line)",41,0,1,33,0,1.0,677.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",9.0,239.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,1407,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
1,1405,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
2,1404,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
3,1402,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
4,1397,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
5,1396,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
6,1394,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
7,1393,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0
8,1389,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_tvb.c,TvbRange__tostring,"(/home/workspace/epan/wslua/wslua_tvb.c, TvbRange__tostring)",1.0,1.0


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,937,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
1,935,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
2,932,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
3,930,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
4,929,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,808,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
68,807,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
69,806,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0
70,805,0,1,33,0,1.0,1.0,5026.0,5026.0,/home/workspace/epan/wslua/wslua_proto_field.c,ProtoField_integer,"(/home/workspace/epan/wslua/wslua_proto_field.c, ProtoField_integer)",1.0,1.0


Top 5:


,meth,vote,rank
0,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",63.993164,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,571,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
1,570,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
2,569,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
3,568,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
4,567,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,335,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
177,330,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
178,328,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
179,327,1,1,32,0,0.970588,0.707107,7696.0,7696.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553


# 감상

Max aggregation으로 top이지만 그게 677개. line 수가 많은 편이 아니어서 voting으로도 성적이 좋지 않음